In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
import matplotlib.pyplot as plt
import numpy as np
import jlc
import scipy
import cv2
from utils import estimate_bias, estimate_bias_coefs,norm_quantile,norm_translation,process_vol
from pathlib import Path
import os

In [6]:
filenames = list(Path("./data/").glob("*.tif"))
filenames = [str(f) for f in filenames]
print(filenames)

['data\\YCsgA_Exp2_Pos1.tif', 'data\\YCsgA_Exp2_Pos2.tif', 'data\\YCsgA_Exp2_Pos3.tif', 'data\\YCsgA_Exp2_Pos4.tif', 'data\\YCsgA_Exp3_Pos1.tif', 'data\\YCsgA_Exp3_Pos2.tif', 'data\\YCsgA_Exp3_Pos3.tif', 'data\\YCsgA_Exp3_Pos4.tif']


In [7]:
#process and save
for filename in filenames:
    savename = filename.replace("data","data_processed")
    os.makedirs(os.path.dirname(savename),exist_ok=True)
    jlc.save_tifvol(process_vol(filename),savename)
    print("done with ",filename)

done with frame 0/414
done with frame 50/414
done with frame 100/414
done with frame 150/414
done with frame 200/414
done with frame 250/414
done with frame 300/414
done with frame 350/414
done with frame 400/414
done with  data\YCsgA_Exp2_Pos1.tif
done with frame 0/414
done with frame 50/414
done with frame 100/414
done with frame 150/414
done with frame 200/414
done with frame 250/414
done with frame 300/414
done with frame 350/414
done with frame 400/414
done with  data\YCsgA_Exp2_Pos2.tif
done with frame 0/414
done with frame 50/414
done with frame 100/414
done with frame 150/414
done with frame 200/414
done with frame 250/414
done with frame 300/414
done with frame 350/414
done with frame 400/414
done with  data\YCsgA_Exp2_Pos3.tif
done with frame 0/414
done with frame 50/414
done with frame 100/414
done with frame 150/414
done with frame 200/414
done with frame 250/414
done with frame 300/414
done with frame 350/414
done with frame 400/414
done with  data\YCsgA_Exp2_Pos4.tif
done

In [4]:
filenames = list(Path("./data_processed/").glob("*.tif"))
filenames = [str(f) for f in filenames]
print(filenames)

['data_processed\\YCsgA_Exp2_Pos1.tif']


In [10]:
vol = jlc.load_tifvol(filenames[2].replace("data","data_processed"))
jlc.inspect_vol(vol.transpose(0,2,1))

In [64]:
class FiberGraph:
    def __init__(self, find_conn_func=None):
        self.nodes = []
        self.edges = []
        self.node_to_cc = []
        self.cc_to_node = []
        self.XY = np.zeros((0,2))
        if find_conn_func is None:
            self.find_conn_func = self.default_find_conn_func
        else:
            self.find_conn_func = find_conn_func
    
    def add_node(self,x,y,timestep,conn=None,is_growing=None):
        node = {"idx": len(self.nodes),
                "x": x,
                "y": y,
                "timestep": timestep,
                "is_growing": True if is_growing is None else is_growing}
        
        if conn is None and self.num_fibers()==0:
            conn = []
        if conn is None:
            dist = np.linalg.norm(self.XY-np.array([x,y]),axis=1)
            t = np.array([n["timestep"] for n in self.nodes])
            conn = self.find_conn_func(dist,timestep,t)
        if isinstance(conn,int):
            if isinstance(conn,int):
                conn = [conn]
        assert isinstance(conn,list)
        
        for c in conn:
            self.add_edge(node["idx"],c)

        node["conn"] = conn
        if len(conn)>0:
            node["root"] = False
            self.node_to_cc.append(self.node_to_cc[conn[0]])
            self.cc_to_node[self.node_to_cc[conn[0]]].append(node["idx"])
        else:
            node["root"] = True
            self.node_to_cc.append(len(self.cc_to_node))
            self.cc_to_node.append([node["idx"]])
        self.nodes.append(node)
        self.XY = np.vstack((self.XY,[x,y]))

    def add_edge(self,idx1,idx2):
        self.edges.append((idx1,idx2))
        self.nodes[idx2]["conn"].append(idx1)
        self.nodes[idx2]["is_growing"] = False

    def __len__(self):
        return len(self.nodes)

    def num_fibers(self):
        return len(self.cc_to_node)
    
    def plot(self,color=None,plot_cc=None):
        if plot_cc is None:
            plot_cc = np.arange(self.num_fibers())
        if color is None:
            color = np.random.rand(self.num_fibers(),3)
        else:
            if isinstance(color,str):
                color = np.array([color]*self.num_fibers())
            elif isinstance(color,(list,np.ndarray)):
                assert len(color)==self.num_fibers(),"color list must have same length as number of fibers if color is array/list"
        for cc_idx in range(len(self.cc_to_node)):
            if cc_idx in plot_cc:
                edges = []
                for node_idx in self.cc_to_node[cc_idx]:
                    node = self.nodes[node_idx]
                    edges += [(node_idx,c) for c in node["conn"]]
                edges_x1_x2_nan = []
                edges_y1_y2_nan = []
                for idx1,idx2 in edges:
                    edges_x1_x2_nan += [self.nodes[idx1]["x"],self.nodes[idx2]["x"],np.nan]
                    edges_y1_y2_nan += [self.nodes[idx1]["y"],self.nodes[idx2]["y"],np.nan]
                plt.plot(edges_x1_x2_nan,edges_y1_y2_nan,".-",color=color[cc_idx])
        XY_growing = self.XY[[n["is_growing"] for n in self.nodes]]
        plt.plot(XY_growing[:,0],XY_growing[:,1],"o",color="green",markerfacecolor='none',linewidth=1)
        XY_root = self.XY[[n["root"] for n in self.nodes]]
        plt.plot(XY_root[:,0],XY_root[:,1],"o",color="red",markerfacecolor='none',markersize=10)

    def default_find_conn_func(self,dist,t_now,t):
        mod_dist = dist+0.1*(t_now-t-1)**3
        if any(mod_dist<5):
            return [np.argmin(mod_dist)]
        else:
            return []
fiber_graph = FiberGraph()
n = 10
for i in range(100):
    fiber_graph.add_node(np.random.rand()*n,np.random.rand()*n,i)


plt.imshow(np.zeros((n,n)),cmap="gray",extent=[0,n,0,n])
fiber_graph.plot()

In [52]:
sum([n["root"] for n in fiber_graph.nodes])

70

In [50]:
plt.hist([len(n["conn"]) for n in fiber_graph.nodes])

(array([55.,  0.,  0.,  0.,  0., 30.,  0.,  0.,  0., 15.]),
 array([0. , 0.2, 0.4, 0.6, 0.8, 1. , 1.2, 1.4, 1.6, 1.8, 2. ]),
 <BarContainer object of 10 artists>)

In [36]:
fiber_graph.cc_to_node

[[0], [1], [2], 3, [4], 5, [6], 7, 8, 9]

In [153]:
def soft_threshold(start=0,stop=1,power=1): 
    to_interval = lambda x: np.clip((x-start)/(stop-start),0,1)
    return lambda x: h(to_interval(x),power)
def h(x,p):
    return (x<0.5)*0.5*2**p*x**p+(x>=0.5)*(1-0.5*2**p*(1-x)**p)

t = np.linspace(-1,2,300)
for p in [0.5,1,2,3]:
    f = soft_threshold(0,1,p)
    plt.plot(t,f(t))

In [155]:
jlc.inspect_vol(soft_threshold(0.1,0.3,2)(vol))

In [205]:
jlc.inspect_vol(np.concatenate((vol, translated_vol), axis=2))

In [31]:
plt.plot(np.array(best_translation)[:,0], label='x')
plt.plot(np.array(best_translation)[:,1], label='y')

In [100]:
from scipy.ndimage import gaussian_filter
smooth_translation = np.array(best_translation)
smooth_translation = gaussian_filter(smooth_translation, sigma=[3,0])
plt.plot(smooth_translation[:,0], label='x')
plt.plot(smooth_translation[:,1], label='y')

In [68]:
plt.plot(np.array(best_translation)[:,0],np.array(best_translation)[:,1])

In [131]:
plt.imshow(vol[:,:,0])

In [132]:
a = gaussian_filter(vol[200,:,0],2)
b = gaussian_filter(vol[205,:,0],2)
plt.scatter(a,b)
plt.plot([0,1],[0,1],color="red")
plt.xlim(min(a.min(),b.min()), max(a.max(),b.max()))
plt.ylim(min(a.min(),b.min()), max(a.max(),b.max()))



(0.021767714915564087, 0.3470474157935479)

In [127]:
b.shape

(684,)

In [139]:
plt.plot(a)
plt.plot(b)

In [128]:
cc = []
for t in translate_search:
    b_translated = cv2.warpAffine(b, np.float32([[1,0,t],[0,1,0]]), (1,b.shape[0]), flags=cv2.INTER_LINEAR,borderMode=cv2.BORDER_REPLICATE)[:,0]
    corr = np.corrcoef(a,b_translated)[0,1]
    cc.append(corr)
plt.plot(translate_search, cc)